In [52]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import re
import sys
import ccobra
from itertools import combinations

In [57]:
def read_mesh(file):
        out = []
        with open(f"{file}", 'r') as f:
            contents = f.read().split('\n')
            for i in range(len(contents)):
                if contents[i].startswith('Name'):
                    match = re.search(r'\".+\"', contents[i])
                    ex_name = match[0].strip('\"').split()
                    ex_name[0] = ex_name[0].replace('_', ' ')
                    for j in range(i,len(contents)):
                        if contents[j].startswith('Input'):
                            match = re.search(r'Target ([0 1]+)', contents[j])
                            ex_target = [int(elem) for elem in match[0].strip('Target ').split()]
                            break
                    out.append((ex_name, ex_target))
        return out

In [58]:
a = read_mesh('dfs_data/syllogisms.mesh')
a = [tup for tup in a if tup[0][-1] != tup[0][-2]]

In [59]:
premisses = [tup[0] for tup in a]
p = [double for double in list(combinations(premisses, 2))]
len(p)

276

In [60]:
for double in p:
    print(double, ccobra.syllogistic.encode_task(double))

(['All', 'trumpettists', 'pianists'], ['All', 'trumpettists', 'singers']) AA4
(['All', 'trumpettists', 'pianists'], ['All', 'pianists', 'trumpettists']) AA1
(['All', 'trumpettists', 'pianists'], ['All', 'pianists', 'singers']) AA1
(['All', 'trumpettists', 'pianists'], ['All', 'singers', 'trumpettists']) AA2
(['All', 'trumpettists', 'pianists'], ['All', 'singers', 'pianists']) AA3
(['All', 'trumpettists', 'pianists'], ['Some', 'trumpettists', 'pianists']) AI4
(['All', 'trumpettists', 'pianists'], ['Some', 'trumpettists', 'singers']) AI4
(['All', 'trumpettists', 'pianists'], ['Some', 'pianists', 'trumpettists']) AI1
(['All', 'trumpettists', 'pianists'], ['Some', 'pianists', 'singers']) AI1
(['All', 'trumpettists', 'pianists'], ['Some', 'singers', 'trumpettists']) AI2
(['All', 'trumpettists', 'pianists'], ['Some', 'singers', 'pianists']) AI3
(['All', 'trumpettists', 'pianists'], ['No', 'trumpettists', 'pianists']) AE4
(['All', 'trumpettists', 'pianists'], ['No', 'trumpettists', 'singers']

In [63]:
ccobra.syllogistic.decode_response('Aac', p[0])

[['All', 'pianists', 'singers']]